## CCTVストリーム画像から災害検知するエージェント
### 🧩 システム構成（MVP）
- graph TD
- A[CCTV画像ストリーム] --> B[画像キャプチャ & 前処理]
- B --> C[UI-TARS-1.5: Image-to-Text]
- C --> D[災害キーワード、災害スコア判定モジュール]
- D --> E[位置特定（カメラID → 地図）]
- E --> F[被害要約生成]
- F --> G[自然言語レポート生成]

#### 1. 📸 画像キャプチャ & 前処理
- 各CCTVから定期的に画像を取得（例：1分ごと）
- 解像度調整、ノイズ除去、タイムスタンプ付与

##### 🔧 必要なツール
- | ツール | 用途 | 
- | ffmpeg | ストリームからフレームを抽出 | 
- | OpenCV | Pythonで画像処理と保存 | 
##### インストール
- pip install yt-dlp opencv-python
- ⇒Successfully installed opencv-python-4.12.0.88 yt-dlp-2025.6.30 
- ▼Not well：sudo apt install ffmpeg
##### sudo無効の仮想環境での対処法
- そのメッセージは、お使いの環境（一部の仮想環境）で sudo 権限が無効化されていることを示しています。以下に、状況別の対処法を整理しました：
- 🛠️ 対処法：sudo が無効な環境で ffmpeg を使うには？
- ✅ 方法①：ffmpeg をバイナリで直接インストール（推奨）
- 公式ffmpegビルドサイト または FFmpeg GitHub から静的バイナリをダウンロード、例（Linux x86_64）:
- 実行ファイルをパスに追加（例：~/.local/bin）
- 動作確認
##### ffmpeg.exe Windows用
- ffmpeg（拡張子なし）は Linux用の実行バイナリ
- Windowsでは .exe 拡張子のある Windowsネイティブビルド が必要
✅ 解決策：Windows向けの正しい ffmpeg.exe を入手する
以下の手順で、Windows用の ffmpeg.exe を確実に取得できます：
🔗 ステップ1：公式Windowsビルドサイトにアクセス
👉 https://www.gyan.dev/ffmpeg/builds/
📦 ステップ2：次のいずれかをダウンロード
- ffmpeg-release-essentials.zip（軽量で必要最小限）、解凍し、exeを確認
✅フォルダーを作成
- New-Item -ItemType Directory -Path "C:\Tools\ffmpeg\bin" -Force
✅環境変数 PATH に追加（永続化）
- $env:Path += ";C:\Tools\ffmpeg\bin"
✅ 動作確認
- ffmpeg -version → バージョン情報が表示されれば成功です！
- ffmpeg version 7.1.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers

### 🎥 ストリーム画像取得スクリプト（Windows環境向け）
CCTVストリームからの画像取得スクリプトの整備に進みましょう。今回は「球磨川YouTubeライブ映像」などを対象に、1分ごとに静止画像を保存できるMVPコードを用意します。

✅ 必要なツール
| ツール | 用途 | 
| yt-dlp | YouTubeライブのストリームURLを取得 | 
| ffmpeg.exe | ストリームから静止画を抽出 | 
| PowerShell | スケジュール＆保存処理 | 

- PowerShellスクリプトをPythonから実行できるように修正しました。
新しいCell In[1]は、subprocessを使ってPowerShellスクリプトを呼び出すPythonコードになっています。
- このまま実行すれば、PowerShellの処理がPythonから開始されます。

In [1]:
import subprocess
powershell_script = r'''
$youtubeURL = "https://www.youtube.com/watch?v=ckuAblc8GCw"
$savePath   = "C:\Users\yasun\PyTorch\CCTVDisasterAgent\KumagawaFrames"
$ffmpegPath = "C:\Tools\ffmpeg\bin\ffmpeg.exe"  # 必要に応じて変更
$interval   = 10  # 60秒
$Num_of_Captures = 3  # 画像取得回数
New-Item -ItemType Directory -Path $savePath -Force | Out-Null
for ($i = 0; $i -lt $Num_of_Captures; $i++) {
    $timestamp = Get-Date -Format "yyyyMMdd_HHmmss"
    $imageFile = "$savePath\frame_$timestamp.jpg"
    $streamURL = yt-dlp -g $youtubeURL
    & "$ffmpegPath" -y -i "$streamURL" -frames:v 1 "$imageFile"
    Write-Host "✅ Saved: $imageFile"
    Start-Sleep -Seconds $interval
}
'''
subprocess.run(['powershell', '-Command', powershell_script])

CompletedProcess(args=['powershell', '-Command', '\n$youtubeURL = "https://www.youtube.com/watch?v=ckuAblc8GCw"\n$savePath   = "C:\\Users\\yasun\\PyTorch\\CCTVDisasterAgent\\KumagawaFrames"\n$ffmpegPath = "C:\\Tools\\ffmpeg\\bin\\ffmpeg.exe"  # 必要に応じて変更\n$interval   = 10  # 60秒\n$Num_of_Captures = 3  # 画像取得回数\nNew-Item -ItemType Directory -Path $savePath -Force | Out-Null\nfor ($i = 0; $i -lt $Num_of_Captures; $i++) {\n    $timestamp = Get-Date -Format "yyyyMMdd_HHmmss"\n    $imageFile = "$savePath\\frame_$timestamp.jpg"\n    $streamURL = yt-dlp -g $youtubeURL\n    & "$ffmpegPath" -y -i "$streamURL" -frames:v 1 "$imageFile"\n    Write-Host "✅ Saved: $imageFile"\n    Start-Sleep -Seconds $interval\n}\n'], returncode=0)

### 2. 🧠 Qwen2.5-LV-3Bによる画像理解(Image-to-Text LLM Prediction)
- 各画像に対して以下のプロンプトを付与：
- <|system|> あなたは災害監視エージェントです。
- <|user|> この画像に洪水の兆候がありますか？水位、道路冠水、土砂崩れなどを説明してください。
- <|vision_start|>...画像トークン...<|vision_end|>
##### ⇒出力：自然言語による被害説明（例：「道路が冠水しており、車両が立ち往生している」）

### method. INT4量子化を使ってQwen2.5-LV-3BのようなLLMを軽量化・高速化する
- 以下の手順で実装できます。計算環境（RTX 4060 Ti 16GB）なら、INT4量子化は非常に有効です。

---

#### ⚙️ INT4量子化の実装ステップ（AutoGPTQを例に）
#### 🧠 INT4量子化のポイント

| 項目 | 内容 |
|------|------|
| 精度 | INT4はFP16より若干精度が落ちるが、実用上問題ないケースが多い |
| メモリ使用量 | 約1/4に削減（FP32比） |
| 推論速度 | 約1.5～2倍高速化 |
| 対応モデル | GPTQ形式で量子化されたモデル（Hugging Faceで「-GPTQ」表記） |

##### ① 必要ライブラリのインストール
- Pythonセルでpip installを実行する場合は、コマンドの前に!を付ける必要があります。

In [ ]:
# Qwen2-VLに必要なライブラリをインストール
!pip install transformers>=4.37.0
!pip install Pillow
!pip install torch torchvision
!pip install accelerate
# Windows環境向けのbitsandbytesインストール
!pip install bitsandbytes --no-deps
!pip install bitsandbytes
!pip install hf_xet  # Hugging Faceからの高速ダウンロード用

  Using cached sentencepiece-0.2.0-cp311-cp311-win_amd64.whl.metadata (8.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.7.0-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.3-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp311-cp311-win_amd64.whl.metadata (76 kB)
   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
   --------- ------------------------------ 1.0/4.6 MB 10.0 MB/s eta 0:00:01
   ------------------ --------------------- 2.1/4.6 MB 5.9 MB/s eta 0:00:

##### ② INT4量子化済みモデルの準備（Hugging Faceから取得）
##### 修正内容1：
- 正しいモデル名: Qwen/Qwen2-VL-2B-Instruct に変更（実際に存在するモデル）
- 適切なライブラリ: Qwen2VLForConditionalGeneration と AutoProcessor を使用
- 推論コード: Qwen2.5-VLの正しい推論フォーマットに変更

##### 修正内容2：
- hf_xetパッケージを追加: Hugging Faceからのダウンロード速度が向上します
- メモリ最適化設定を追加:
- low_cpu_mem_usage=True: CPUメモリ使用量を削減
- use_cache=True: キャッシュを有効化
- VRAM使用量の表示を追加

##### これらの設定により：
- ダウンロード速度が向上
- メモリ使用量が最適化される
- VRAM使用量を監視できる

In [ ]:
# !pip install hf_xet

  Using cached hf_xet-1.1.5-cp37-abi3-win_amd64.whl.metadata (883 bytes)
Using cached hf_xet-1.1.5-cp37-abi3-win_amd64.whl (2.7 MB)


##### 修正内容3：
- モデル名を修正: Qwen/Qwen2-VL-2B-Instruct に変更（より安定で軽量）
- 4bit量子化を追加: BitsAndBytesConfigを使ってVRAM使用量を大幅削減
- bitsandbytesライブラリを追加: 量子化に必要なライブラリ
###### これらの変更により：
-VRAM使用量が約1/2に削減（約6GBになった）
- モデルの互換性問題が解決
- より安定した動作が期待できます
- 量子化により若干の精度低下はありますが、災害検知には十分な性能を維持できるはずです。

##### 修正内容4：
- bitsandbytesの確実なインストール: --no-deps オプションを使って依存関係の問題を回避
- フォールバック機能: bitsandbytesが利用できない場合は通常のBF16モードで動作
- エラーハンドリング: ImportErrorをキャッチして適切に処理
##### これで以下のような動作になります：
- bitsandbytesが正常にインストールされている場合 → 4bit量子化で動作（VRAM約6GB）
- bitsandbytesが利用できない場合 → 通常のBF16モードで動作（VRAM約8-12GB）

In [ ]:
#!pip install bitsandbytes

   ---------------------------------------- 0.0/72.2 MB ? eta -:--:--
    --------------------------------------- 1.0/72.2 MB 7.2 MB/s eta 0:00:10
   - -------------------------------------- 2.6/72.2 MB 6.9 MB/s eta 0:00:11
   -- ------------------------------------- 3.7/72.2 MB 6.2 MB/s eta 0:00:11
   --- ------------------------------------ 6.3/72.2 MB 7.7 MB/s eta 0:00:09
   ---- ----------------------------------- 8.7/72.2 MB 8.5 MB/s eta 0:00:08
   ------ --------------------------------- 11.3/72.2 MB 9.2 MB/s eta 0:00:07
   ------- -------------------------------- 13.9/72.2 MB 9.6 MB/s eta 0:00:07
   --------- ------------------------------ 16.5/72.2 MB 9.8 MB/s eta 0:00:06
   ---------- ----------------------------- 19.1/72.2 MB 10.1 MB/s eta 0:00:06
   ----------- ---------------------------- 21.5/72.2 MB 10.2 MB/s eta 0:00:05
   ------------- -------------------------- 24.1/72.2 MB 10.4 MB/s eta 0:00:05
   -------------- ------------------------- 26.7/72.2 MB 10.5 MB/s eta 0:0

In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import torch

# 正しいQwen2-VLモデル名に修正
model_name = "Qwen/Qwen2-VL-2B-Instruct"  # 2Bモデルに変更（より安定）

print(f"📥 モデル {model_name} をダウンロード中...")

# bitsandbytesが利用可能かチェック
try:
    from transformers import BitsAndBytesConfig
    
    # 4bit量子化設定でVRAM使用量を大幅削減
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    # モデル読み込み（量子化あり）
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=quantization_config
    )
    print("✅ 4bit量子化でモデルを読み込みました")
    
except ImportError:
    print("⚠️ bitsandbytesが利用できません。通常モードで読み込みます")
    # 量子化なしでのモデル読み込み
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
    print("✅ 通常モード（BF16）でモデルを読み込みました")

# プロセッサーの読み込み
processor = AutoProcessor.from_pretrained(model_name)

print(f"✅ モデル {model_name} の読み込み完了")
print(f"💾 VRAM使用量: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

c:\Users\yasun\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📥 モデル Qwen/Qwen2-VL-2B-Instruct をダウンロード中...


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.89s/it]


✅ 4bit量子化でモデルを読み込みました


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


✅ モデル Qwen/Qwen2-VL-2B-Instruct の読み込み完了
💾 VRAM使用量: 1.42 GB


##### ③ 推論の実行(Qwen2-VL-2B-Instruct)
- コンセプトは、CCTV画像を10分ごとに取得し、LLMにより災害を検知すること。
- 推論の動作確認においては、対象画像のファイル名を入力して、推論を実行した。
- 災害の特徴を質問するpromptは、英語を用いた。後に、英語のキーワードを抽出するため。
##### テスト画像と推論実行時間
- ケース１：雨の後で洪水でないCCTV画像、size FHD 1920x1080、推論時間10～11.7秒/枚
- ケース２：実際の河川氾濫画像（ドローン画像）、size 1000x525, 1024x683, 1200x630、推論時間28.3～29.5秒/枚
##### 推論時間の考察
- 実際の河川氾濫画像では、画像サイズがより小さいにも関わらず、災害の特徴量を詳細に検出しており、推論時間に3倍を要した。
- 一方、河川が濁流であっても、氾濫していないCCTV画像は、あっさりと推論時間が短かった。

In [ ]:
# Qwen2-VL-2B-Instructを使った画像解析の推論
from PIL import Image

# 画像パスを設定
image_path = r"C:\Users\yasun\PyTorch\CCTVDisasterAgent\8_TestAfterRain\frame_20250716_090940.jpg"

# 画像を読み込み
image = Image.open(image_path)

# プロンプトを設定
prompt = "Please analyze this image and describe any signs of flooding, road damage, or disaster impact."

# 会話形式のメッセージ
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image,
            },
            {"type": "text", "text": prompt},
        ],
    }
]

# プロンプトをテキストに変換
text_prompt = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

# 入力を処理
inputs = processor(
    text=text_prompt,
    images=image,
    return_tensors="pt"
)

# GPUに移動
inputs = {k: v.to("cuda") for k, v in inputs.items()}

# 推論実行
with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7
    )

# デコード
generated_text = processor.batch_decode(
    generated_ids[:, inputs["input_ids"].shape[1]:], 
    skip_special_tokens=True
)[0]

print("🧠 Qwen2-VL 分析結果:")
print(generated_text)

# 後続の処理のために変数を保存
description_text = generated_text

🧠 Qwen2-VL 分析結果:
The image shows a bridge over a river, with a calm water surface and no visible signs of flooding or road damage. The bridge appears to be in good condition, and there are no visible signs of distress or damage. The surrounding area is lush with greenery, indicating that the area is likely well-maintained and not affected by natural disasters. The weather appears to be overcast, but there are no visible signs of precipitation or storm activity.


### 3. 🚨 災害キーワード抽出、災害スコア判定モジュール
- 出力文から災害関連キーワードを抽出（例：冠水、氾濫、土砂）
- スコアリングして「災害あり／なし」を判定

In [31]:
def score_flood_claude(description: str) -> float:
    keywords = {
        "flood": 0.4,
        "overflow": 0.3,
        "impassable": 0.4,
        "damage": 0.3,
        "debris": 0.2,
        "storm": 0.2,
        "high water": 0.3,
        "water level": 0.3,
        "bridge failure": 0.5,
        "road washed out": 0.5
    }

    score = 0.0
    lowered = description.lower()

    # キーワードごとにスコアを加算
    for word, weight in keywords.items():
        if word in lowered:
            score += weight
    return min(score, 3.0)

# 該当したキーワード抽出してリスト化
def extract_keywords(description: str) -> list:
    keywords = {
        "flood": 0.4,
        "overflow": 0.3,
        "impassable": 0.4,
        "damage": 0.3,
        "debris": 0.2,
        "storm": 0.2,
        "high water": 0.3,
        "water level": 0.3,
        "bridge failure": 0.5,
        "road washed out": 0.5
    }

    found_keywords = []
    lowered = description.lower()

    for word in keywords.keys():
        if word in lowered:
            found_keywords.append(word)

    return found_keywords

# Qwen2-VLの出力を使って被害説明の抽出とスコア計算、キーワードリストを表示
print("📜 被害説明:", description_text)

score = score_flood_claude(description_text)
print(f"📊 災害スコア: {score:.2f}")
found_disaster_words = extract_keywords(description_text)
print("🔍 該当キーワード:", found_disaster_words)

📜 被害説明: The image shows a bridge over a river, with a calm water surface and no visible signs of flooding or road damage. The bridge appears to be in good condition, and there are no visible signs of distress or damage. The surrounding area is lush with greenery, indicating that the area is likely well-maintained and not affected by natural disasters. The weather appears to be overcast, but there are no visible signs of precipitation or storm activity.
📊 災害スコア: 0.90
🔍 該当キーワード: ['flood', 'damage', 'storm']


#### JSON形式で出力

In [32]:
# JSON形式でのレスポンスを保存
import json

# found_disaster_wordsをテキスト化
import os
found_disaster_words_text = ", ".join(found_disaster_words) if found_disaster_words else "None"

response_data = {
    "description": description_text,
    "score": score,
    "found_disaster_words": found_disaster_words_text
}   

#### 5. 📝 被害要約生成
- 被害内容を要約し、以下のようなレポートを生成：
- 【災害報告】
- 主な災害事象：キーワード
- 災害スコア：1.3
- 災害を検知した画像：ファイル名
- 検知時刻：2025年7月9日 06:15
- 状況：道路が冠水し、車両が通行不能。水位は歩道を越えている。
- 詳細な被害説明：画像から推論出力の全文

In [33]:
from datetime import datetime

def generate_disaster_report(description, keywords, score, image_filename, detected_time,description_text):
    # スコアの正規化
    normalized_score = min(score, 1.0)

    # フォーマット用の時刻変換（例: "20250709_0615" → "2025年7月9日 06:15"）
    def format_timestamp(ts_str):
        try:
            dt = datetime.strptime(ts_str, "%Y%m%d_%H%M")
            return dt.strftime("%Y年%m月%d日 %H:%M")
        except:
            return detected_time  # 変換できなければそのまま使う

    # レポート本文の生成
    report = f"""
【災害報告】
- 主な災害事象：{keywords}
- 災害スコア：{normalized_score:.1f}
- 災害を検知した画像：{image_filename}
- 検知時刻：{format_timestamp(image_filename.split("frame_")[-1].split(".jpg")[0])}
- 状況：{description}
- 詳細な推論と説明：{description_text}
""".strip()

    return report

# レポート生成＆表示
report_text = generate_disaster_report(
    description=response_data["description"],
    keywords=response_data["found_disaster_words"],
    score=response_data["score"],
    image_filename=os.path.basename(image_path),
    detected_time="2025年7月9日 06:15",
    description_text=description_text
)

print(report_text)

#　report_textをJSON形式で出力
import json
report_json = {
    "report": report_text,
    "image_filename": os.path.basename(image_path),
    # "detected_time": "2025年7月9日 06:15"
}
#   保存するファイル名に、画像ファイル名を含める
report_filename = f"disaster_report_{os.path.basename(image_path).split('.')[0]}.json"
with open(report_filename, "w", encoding="utf-8") as f:
    json.dump(report_json, f, ensure_ascii=False, indent=4)
print(f"📝 レポートを保存しました: {report_filename}")


【災害報告】
- 主な災害事象：flood, damage, storm
- 災害スコア：0.9
- 災害を検知した画像：frame_20250716_090940.jpg
- 検知時刻：2025年7月9日 06:15
- 状況：The image shows a bridge over a river, with a calm water surface and no visible signs of flooding or road damage. The bridge appears to be in good condition, and there are no visible signs of distress or damage. The surrounding area is lush with greenery, indicating that the area is likely well-maintained and not affected by natural disasters. The weather appears to be overcast, but there are no visible signs of precipitation or storm activity.
- 詳細な推論と説明：The image shows a bridge over a river, with a calm water surface and no visible signs of flooding or road damage. The bridge appears to be in good condition, and there are no visible signs of distress or damage. The surrounding area is lush with greenery, indicating that the area is likely well-maintained and not affected by natural disasters. The weather appears to be overcast, but there are no visible signs of precipita

#### Note1.**GPUクラッシュがいくつかの要因で複合発生した可能性**

---

##### 🧠 モデルとGPU構成のマッチング評価

| 項目 | 条件 | 影響 |
|------|------|------|
| モデルサイズ | Qwen2.5-LV-3B（約3Bパラメータ） | 中規模モデル |
| 推論形式 | 画像入力あり（1280×720 HD） | メモリ消費増加（特にマルチモーダル） |
| GPU | ASUS RTX 4060 Ti 16GB GDDR6 | VRAMは十分。ただしメモリ帯域はミッドレンジ |

---

##### ⚠️ 推論クラッシュの可能性要因

1. **マルチモーダル推論のVRAM消費量**
   - Qwen2.5-LVは画像も扱うため、**画像埋め込み・Transformer処理でVRAMを大量に消費**します。
   - HD画像1枚でも内部でトークンに分割後、**最大128Kコンテキスト長を許容する構造**のため、メモリ爆発が起こる可能性があります。

2. **FlashAttention未使用 / Efficient Layer未導入**
   - メモリ節約技術（例：FlashAttention2、PagedAttention、xFormers）を使っていない場合、**VRAMが急激に枯渇**します。

3. **16GB VRAMの内訳とOS使用分**
   - 実際に使えるのは14～15GB程度。さらに**CUDAカーネルやドライバ、XLAのオーバーヘッド**が入ると残りが圧迫されます。

---

##### ✅ 対処・改善案

| 方法 | 具体策 | 利点 |
|------|--------|------|
| 量子化 | INT4/Q8モデルに切替（例：AutoGPTQ, bitsandbytes） | メモリ節約、推論高速化 |
| コンテキスト制限 | 入力長を256～1024程度に制限 | VRAM消費大幅減 |
| 高速Attention | FlashAttention2やPagedAttentionの導入 | メモリ効率化・速度改善 |
| バッチサイズ | 単一推論（batch=1）にする | メモリ爆発の回避 |
| 推論フレームワーク | vLLMやOpenVINOなど軽量推論ライブラリを使用 | VRAM最適化、クラッシュ防止 |

---
##### ✅ 実行結果
- 実際に、**テスト画像2に対して、Qwen2-VL-2B-Instructを用いて推論した結果、6GBの容量を必要**とした。
---


#### Note2. **VRAM（Video RAM）とは、GPU専用の高速メモリ**
- 画像や映像などのグラフィックスデータを一時的に保存・処理するために使われます。CPUが使うRAMとは異なり、GPUが描画処理を効率よく行うための作業領域です。

---

##### 🔍 VRAMの役割と特徴

- **GPUが扱う画像・映像・テクスチャなどを一時保存**
- **高解像度・高フレームレートの描画に不可欠**
- **ゲーム、動画編集、AI推論などで使用量が増加**
- **GDDR6やGDDR7など、RAMより高速な規格を採用**

---

##### 🧠 VRAMとRAMの違い

| 項目 | VRAM | RAM（メインメモリ） |
|------|------|----------------------|
| 用途 | GPUの画像・映像処理 | CPUの一般的な処理 |
| 接続先 | グラフィックボード | マザーボード |
| 処理対象 | テクスチャ、フレームバッファなど | OS、アプリ、データ処理など |
| 規格 | GDDR6, GDDR7など | DDR4, DDR5など |

---

##### ⚠️ VRAM不足で起こる現象

- テクスチャが表示されない、画面がカクつく
- フレームレート低下（fps drop）
- 最悪の場合、アプリやゲームがクラッシュ

---

VRAMは、GPUの性能を引き出すための「専用作業机」のようなもの。容量が多いほど、複雑で高精細な処理がスムーズになります。  
AI推論や最適化処理を行う場合も、モデルサイズやコンテキスト長に応じてVRAMの確保が重要になります。